In [9]:
affect_intensity_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/" + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"

In [10]:
from nltk.corpus import stopwords

def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)

In [11]:
import re

def clean_str(string):
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string) # removing any twitter handle mentions
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return remove_stopwords(string.strip().lower())

In [12]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [27]:
def read_training_data(training_data_file_path):

    train_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], clean_str(array[1]), array[2], float(array[3])))
    return train_list
            
def read_test_data(training_data_file_path):

    test_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            test_list.append(Tweet(array[0], clean_str(array[1]), array[2], None))
    return test_list

In [14]:
emotion = "anger"

In [15]:
training_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.train.txt"
test_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.dev.target.txt"
predictions_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/predictions/" + \
    emotion + "-pred.txt"

In [25]:
word_list = list()
word_intensities = dict()

with open(affect_intensity_file_path) as affect_intensity_file:
    for line in affect_intensity_file:
        word_int_array = line.replace("\n", "").split("\t")
        
        if (word_int_array[2] == emotion):
            word_list.append(word_int_array[0])
            word_intensities[word_int_array[0]] = word_int_array[1]
            
print(len(word_list))

1483


In [30]:
print(word_intensities['outraged'])

0.964


In [35]:
training_tweets = read_training_data(training_data_file_path)

score_train = list()
tweet_train = list()
for tweet in training_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))
    
print(len(score_train))

857


In [36]:
x_train = list()

for tweet in tweet_train:
    x_vector = list()
    for word in word_list:
        if word in tweet:
            x_vector.append(word_intensities[word])
        else:
            x_vector.append(0.0)
    x_train.append(x_vector)

print(len(x_train))

857


In [40]:
from sklearn import model_selection, ensemble

scores = \
    model_selection.cross_val_score(
        sklearn.ensemble.GradientBoostingRegressor(), x_train, 
        score_train, cv=10, scoring='r2'
    )

mean_score = scores.mean()
print("Accuracy: %0.2f (+/- %0.2f)" % (mean_score, scores.std() * 2))

Accuracy: -11.85 (+/- 10.59)


In [42]:
test_tweets = read_test_data(test_data_file_path)
tweet_test = list()
for tweet in test_tweets:
    tweet_test.append(tweet.text)

print(len(tweet_test))

x_test = list()

for tweet in tweet_test:
    x_vector = list()
    for word in word_list:
        if word in tweet:
            x_vector.append(word_intensities[word])
        else:
            x_vector.append(0.0)
    x_test.append(x_vector)

print(len(x_test))

84
84


In [43]:
ml_model = sklearn.ensemble.GradientBoostingRegressor()
ml_model.fit(x_train, score_train)

y_test = ml_model.predict(X=x_test)

In [44]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )